In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

year = 2022

In [2]:
format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}','profit':'{:,.2f}',
              'pct':'{:,.2f}%','net':'{:,.2f}',
              'cost_amt':'{:,.2f}','unit_cost':'{:,.2f}','avg_cost':'{:,.2f}'}

pd.set_option('display.float_format','{:,.2f}'.format)

### Record selection for transactions

In [3]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month,  
             YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
sells.price AS sell_price, buys.price AS buy_price, 
(sells.price - buys.price) AS diff, qty, 
(sells.price * qty) AS sell_amt,
(buys.price * qty) AS buy_amt,
(sells.price - buys.price) * qty AS gross, 
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s
ORDER BY sell_month DESC, name'''

sql = sql % year
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month,  
             YEAR(buys.date) AS buy_year, MONTH(buys.date) AS buy_month,
sells.price AS sell_price, buys.price AS buy_price, 
(sells.price - buys.price) AS diff, qty, 
(sells.price * qty) AS sell_amt,
(buys.price * qty) AS buy_amt,
(sells.price - buys.price) * qty AS gross, 
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, profit, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2022
ORDER BY sell_month DESC, name


In [4]:
sells_df.dtypes

name           object
sell_year       int64
sell_month      int64
buy_year        int64
buy_month       int64
sell_price    float64
buy_price     float64
diff          float64
qty             int64
sell_amt      float64
buy_amt       float64
gross         float64
pct           float64
profit        float64
kind           object
dtype: object

In [5]:
sells_df.groupby(['sell_year','sell_month','buy_year','buy_month','name']).gross.sum()

sell_year  sell_month  buy_year  buy_month  name  
2022       1           2019      11         MCS      -10,500.00
                       2021      3          IVL        7,500.00
                                 6          IVL       25,500.00
                                 7          IVL       18,750.00
                                            SAT        8,250.00
                                 9          DCC        4,000.00
                                            EPG        2,200.00
                                            NER       -1,350.00
                                 10         SAT       10,500.00
                                 11         KBANK     12,192.00
                                            RCL        7,500.00
                                            SIS        3,375.00
                                            TOP        5,500.00
                                 12         IMH       26,500.00
                                            KBANK    

In [6]:
sold_grp = sells_df.groupby(['name'])
sold_stocks = sold_grp['sell_amt','buy_amt','qty','gross'].sum()
sold_stocks.sort_values(['name'],ascending=[True]).style.format(format_dict)

C:\Users\User\AppData\Local\Temp/ipykernel_2476/2967423552.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sold_stocks = sold_grp['sell_amt','buy_amt','qty','gross'].sum()


,sell_amt,buy_amt,qty,gross
name,,,,
BGRIM,"297,000.00","360,000.00","9,000","-63,000.00"
CKP,"157,000.00","150,000.00","30,000","7,000.00"
DCC,"120,000.00","116,000.00","40,000","4,000.00"
DIF,"143,000.00","147,000.00","10,000","-4,000.00"
EPG,"271,800.00","284,000.00","26,000","-12,200.00"
GLOBAL,"315,000.00","291,000.00","15,000","24,000.00"
IMH,"357,300.00","308,000.00","19,000","49,300.00"
IP,"66,000.00","60,600.00","3,000","5,400.00"
IVL,"835,350.00","756,000.00","18,000","79,350.00"


In [7]:
sold_stocks['sell_price'] = sold_stocks['sell_amt'] / sold_stocks['qty']
sold_stocks['buy_price'] = sold_stocks['buy_amt'] / sold_stocks['qty']
cols = 'sell_amt buy_amt gross qty sell_price buy_price'.split()
sold_stocks[cols].sort_values(['name'],ascending=[True]).style.format(format_dict)

,sell_amt,buy_amt,gross,qty,sell_price,buy_price
name,,,,,,
BGRIM,"297,000.00","360,000.00","-63,000.00","9,000",33.00,40.00
CKP,"157,000.00","150,000.00","7,000.00","30,000",5.23,5.00
DCC,"120,000.00","116,000.00","4,000.00","40,000",3.00,2.90
DIF,"143,000.00","147,000.00","-4,000.00","10,000",14.30,14.70
EPG,"271,800.00","284,000.00","-12,200.00","26,000",10.45,10.92
GLOBAL,"315,000.00","291,000.00","24,000.00","15,000",21.00,19.40
IMH,"357,300.00","308,000.00","49,300.00","19,000",18.81,16.21
IP,"66,000.00","60,600.00","5,400.00","3,000",22.00,20.20
IVL,"835,350.00","756,000.00","79,350.00","18,000",46.41,42.00


### Record selection for active stocks

In [8]:
sql = '''
SELECT name, YEAR(buys.date) AS year, MONTH(buys.date) AS month,
buys.price AS unit_cost, qty, 
(buys.price * qty) AS cost_amt
FROM buys
JOIN stocks ON buys.stock_id = stocks.id
WHERE status = 'Active'
ORDER BY name, buys.date'''
buys_df = pd.read_sql(sql, conpf)
buys_df.style.format(format_dict)

,name,year,month,unit_cost,qty,cost_amt
0,BCH,2021,9,21.70,"6,000","130,200.00"
1,BCH,2021,9,21.30,"6,000","127,800.00"
2,BCH,2021,12,20.50,"3,000","61,500.00"
3,CPNCG,2022,2,12.50,"10,000","125,000.00"
4,DIF,2020,8,14.70,"10,000","147,000.00"
5,DIF,2020,10,14.70,"10,000","147,000.00"
6,DIF,2021,2,14.70,"10,000","147,000.00"
7,DIF,2021,3,14.70,"20,000","294,000.00"
8,DOHOME,2021,8,25.00,"8,000","200,000.00"
9,DOHOME,2022,1,22.90,"4,000","91,600.00"


In [9]:
buys_df.groupby(['year','month','name']).cost_amt.sum()

year  month  name 
2016  9      MCS     167,000.00
      10     MCS     334,000.00
      11     MCS     334,000.00
2017  4      MCS     167,000.00
2018  5      JASIF   300,000.00
                        ...    
2022  2      SIS      60,750.00
             SNC     127,500.00
             SPALI   198,000.00
             SYNEX   290,250.00
             TISCO   196,000.00
Name: cost_amt, Length: 67, dtype: float64

In [10]:
buys_df.groupby(['year','month']).cost_amt.sum()

year  month
2016  9         167,000.00
      10        334,000.00
      11        334,000.00
2017  4         167,000.00
2018  5         300,000.00
2019  11        300,000.00
2020  2         100,000.00
      8         147,000.00
      10        147,000.00
      11        300,000.00
2021  2         147,000.00
      3         616,950.00
      4         165,750.00
      6         200,000.00
      7          94,375.00
      8         363,500.00
      9       1,079,050.00
      10        554,775.00
      11        530,600.00
      12        882,250.00
2022  1       1,114,700.00
      2       2,443,075.00
Name: cost_amt, dtype: float64

In [11]:
buys_df.cost_amt.sum()

10488025.0

In [12]:
buys_df.groupby(['name'])['cost_amt','qty'].sum().style.format(format_dict)

C:\Users\User\AppData\Local\Temp/ipykernel_2476/3973236850.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  buys_df.groupby(['name'])['cost_amt','qty'].sum().style.format(format_dict)


,cost_amt,qty
name,,
BCH,"319,500.00","15,000"
CPNCG,"125,000.00","10,000"
DIF,"735,000.00","50,000"
DOHOME,"291,600.00","12,000"
HREIT,"358,000.00","40,000"
IP,"151,500.00","7,500"
JASIF,"1,000,000.00","100,000"
KCE,"772,500.00","10,000"
LPF,"140,000.00","10,000"


In [13]:
buys_grp = buys_df.groupby(by=['name'])
dtd_stocks = buys_grp['cost_amt','qty'].sum()
dtd_stocks['avg_cost'] = dtd_stocks['cost_amt'] / dtd_stocks['qty']
dtd_stocks.sort_values(['name'],ascending=[True]).style.format(format_dict)

C:\Users\User\AppData\Local\Temp/ipykernel_2476/726847341.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dtd_stocks = buys_grp['cost_amt','qty'].sum()


,cost_amt,qty,avg_cost
name,,,
BCH,"319,500.00","15,000",21.30
CPNCG,"125,000.00","10,000",12.50
DIF,"735,000.00","50,000",14.70
DOHOME,"291,600.00","12,000",24.30
HREIT,"358,000.00","40,000",8.95
IP,"151,500.00","7,500",20.20
JASIF,"1,000,000.00","100,000",10.00
KCE,"772,500.00","10,000",77.25
LPF,"140,000.00","10,000",14.00


In [14]:
file_name = 'unit-cost.csv'
data_file = data_path + file_name
csv_file = csv_path + file_name
box_file = box_path + file_name
data_file, csv_file, box_file

('../data/unit-cost.csv',
 '\\Users\\User\\iCloudDrive\\unit-cost.csv',
 '\\Users\\User\\Dropbox\\unit-cost.csv')

In [15]:
dtd_stocks.sort_values(['name'],ascending=[True]).to_csv(csv_file)
dtd_stocks.sort_values(['name'],ascending=[True]).to_csv(box_file)
dtd_stocks.sort_values(['name'],ascending=[True]).to_csv(data_file)

### Extra addition

In [16]:
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, sells.date AS sell_date,
qty, sells.price AS sell_price, buys.price AS buy_price,
sells.net AS sell_amt, buys.net AS buy_amt, profit,
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, sells.id AS sell_id, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s
ORDER BY sells.date, name'''

sql = sql % year
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, sells.date AS sell_date,
qty, sells.price AS sell_price, buys.price AS buy_price,
sells.net AS sell_amt, buys.net AS buy_amt, profit,
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, sells.id AS sell_id, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2022
ORDER BY sells.date, name


In [17]:
df_sells = pd.read_sql(sql, conpf)
df_sells.head()

,name,sell_year,sell_month,sell_date,qty,sell_price,buy_price,sell_amt,buy_amt,profit,pct,sell_id,kind
0,IVL,2022,1,2022-01-05,3000,44.50,42.00,"133,204.31","126,279.08","6,925.23",5.95,895,DTD
1,DCC,2022,1,2022-01-06,40000,3.00,2.90,"119,734.21","116,256.93","3,477.28",3.45,896,DTD
2,EPG,2022,1,2022-01-06,2000,12.30,11.20,"24,545.52","22,449.62","2,095.90",9.82,897,DTD
3,IMH,2022,1,2022-01-07,1500,16.80,11.00,"25,144.19","16,536.55","8,607.64",52.73,898,DTD
4,IMH,2022,1,2022-01-10,1500,16.80,11.00,"25,144.19","16,536.55","8,607.64",52.73,899,DTD


In [18]:
ttl_by_month = df_sells.groupby(['sell_month'])['profit'].sum()
ttl_by_month

sell_month
1   117,432.48
2   -82,336.60
Name: profit, dtype: float64

In [19]:
ttl_by_month = df_sells.groupby(['sell_year','sell_month','name'], as_index=True).agg(
    {
        'profit':['sum','count'],
    }
)
ttl_by_month

profit      
                                   sum count
sell_year sell_month name                   
2022      1          DCC      3,477.28     1
                     EPG      2,095.90     1
                     IMH     26,377.29     4
                     IVL     49,402.77     4
                     KBANK   13,200.23     3
                     MCS    -10,846.64     1
                     NER     -1,644.03     1
                     RCL      7,187.70     1
                     SAT     18,007.45     2
                     SIS      3,098.42     1
                     SYNEX    2,035.70     1
                     TOP      5,040.41     1
          2          BGRIM  -64,455.19     1
                     CKP      6,320.02     2
                     DIF     -4,642.32     1
                     EPG    -15,526.94     1
                     GLOBAL  22,657.77     2
                     IMH     21,449.14     3
                     IP       5,119.60     1
                     IVL     26,422.57     3
                     MCS    -10,347.75     1
                     NOBLE  -64,814.00     1
                     RATCH      140.34     1
                     TOP     12,575.28     2
                     TSE     -4,451.84     1
                     TSTH    -5,736.72     2
                     TU      -7,046.56     1

In [20]:
pd.set_option('max_rows',None)
ttl_by_month.get('profit')

sum  count
sell_year sell_month name                    
2022      1          DCC      3,477.28      1
                     EPG      2,095.90      1
                     IMH     26,377.29      4
                     IVL     49,402.77      4
                     KBANK   13,200.23      3
                     MCS    -10,846.64      1
                     NER     -1,644.03      1
                     RCL      7,187.70      1
                     SAT     18,007.45      2
                     SIS      3,098.42      1
                     SYNEX    2,035.70      1
                     TOP      5,040.41      1
          2          BGRIM  -64,455.19      1
                     CKP      6,320.02      2
                     DIF     -4,642.32      1
                     EPG    -15,526.94      1
                     GLOBAL  22,657.77      2
                     IMH     21,449.14      3
                     IP       5,119.60      1
                     IVL     26,422.57      3
                     MCS    -10,347.75      1
                     NOBLE  -64,814.00      1
                     RATCH      140.34      1
                     TOP     12,575.28      2
                     TSE     -4,451.84      1
                     TSTH    -5,736.72      2
                     TU      -7,046.56      1

In [21]:
ttl_by_month.groupby(level='name').sum()

profit      
              sum count
name                   
BGRIM  -64,455.19     1
CKP      6,320.02     2
DCC      3,477.28     1
DIF     -4,642.32     1
EPG    -13,431.04     2
GLOBAL  22,657.77     2
IMH     47,826.43     7
IP       5,119.60     1
IVL     75,825.34     7
KBANK   13,200.23     3
MCS    -21,194.39     2
NER     -1,644.03     1
NOBLE  -64,814.00     1
RATCH      140.34     1
RCL      7,187.70     1
SAT     18,007.45     2
SIS      3,098.42     1
SYNEX    2,035.70     1
TOP     17,615.69     3
TSE     -4,451.84     1
TSTH    -5,736.72     2
TU      -7,046.56     1

In [22]:
ttl_by_month.sum()

profit  sum     35,095.88
        count       44.00
dtype: float64

### Profit by Stock

In [23]:
name = 'IMH'
sql = '''
SELECT name, sells.date AS sell_date,
qty, sells.price AS sell_price, buys.price AS buy_price,
sells.net AS sell_amt, buys.net AS buy_amt, profit,
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, sells.id AS sell_id, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s AND name = '%s'
ORDER BY sells.date DESC'''

sql = sql % (year, name)
print(sql)

df_name = pd.read_sql(sql, conpf)
df_name.style.format(format_dict)



SELECT name, sells.date AS sell_date,
qty, sells.price AS sell_price, buys.price AS buy_price,
sells.net AS sell_amt, buys.net AS buy_amt, profit,
ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, sells.id AS sell_id, buys.kind
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2022 AND name = 'IMH'
ORDER BY sells.date DESC


,name,sell_date,qty,sell_price,buy_price,sell_amt,buy_amt,profit,pct,sell_id,kind
0,IMH,2022-02-11,"3,000",18.40,17.60,"55,077.74","52,916.95","2,160.79",4.55%,923,DTD
1,IMH,2022-02-10,"3,000",21.10,17.60,"63,159.80","52,916.95","10,242.85",19.89%,922,DTD
2,IMH,2022-02-09,"3,000",20.70,17.60,"61,962.45","52,916.95","9,045.50",17.61%,921,DTD
3,IMH,2022-01-28,"6,000",17.70,17.60,"105,964.78","105,833.89",130.89,0.57%,913,DTD
4,IMH,2022-01-11,"1,000",20.10,11.00,"20,055.48","11,024.36","9,031.12",82.73%,901,DTD
5,IMH,2022-01-10,"1,500",16.80,11.00,"25,144.19","16,536.55","8,607.64",52.73%,899,DTD
6,IMH,2022-01-07,"1,500",16.80,11.00,"25,144.19","16,536.55","8,607.64",52.73%,898,DTD


In [24]:
profit = df_name.profit.sum()
cost = df_name.buy_amt.sum()
percent = round(profit/cost*100,2)
profit, percent

(47826.43, 15.49)

In [25]:
df_by_price = df_name.groupby(['sell_price'], as_index=True).agg(
    {
        'sell_price':['count'],
    }
)
df_by_price

,sell_price
,count
sell_price,
16.80,2
17.70,1
18.40,1
20.10,1
20.70,1
21.10,1


In [26]:
df_by_price = df_name.groupby(['sell_price','buy_price'], as_index=True).agg(
    {
        'sell_price':['count'],
    }
)
df_by_price

,,sell_price
,,count
sell_price,buy_price,
16.80,11.00,2
17.70,17.60,1
18.40,17.60,1
20.10,11.00,1
20.70,17.60,1
21.10,17.60,1
